# ANALYSER

In [2]:
import Modele
from Var import *
import csv
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import statistics

In [4]:
# Temps exceution mouvement
def temps_exe(duree,route=route_etude,direction=direction_etude,traffic=trafic_etude):
    
    for temps in tqdm(range(duree)):
        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)
    
    return traffic

temps_exe(100000)

 69%|██████▉   | 68776/100000 [00:10<00:04, 6813.66it/s]


KeyboardInterrupt: 

In [8]:
# Idée de test 1 : nbr de voiture total sur la route hors fin


def nbr_voiture(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res


def test_1(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res


r = test_1(route_etude, direction_etude, traffic_etude, 30, 60, 10, 240)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 30 for i in index])

100%|██████████| 30/30 [00:06<00:00,  4.85it/s]

[1474258, 1476362, 1468644, 1464551, 1442019, 1450296, 1453560, 1459681, 1462426, 1465756, 1473313, 1468478, 1484513, 1489224, 1485628, 1477476, 1459597, 1444202, 1448107, 1459273, 1464874, 1476385, 1473374, 1478535, 1490553, 1507941, 1528435, 1538740, 1545906, 1545637]
[34, 47, 48, 35, 36, 49, 46, 37, 38, 33, 50, 39, 41, 32, 40, 52, 30, 31, 51, 45, 53, 42, 44, 43, 54, 55, 56, 57, 59, 58]


Résultat : Test naif qui a pour objectif de vider au plus vite les routes sans vraiment vouloir minimiser les embouteillage. On obtient en général un temps optimal le plus faible étrange !

In [7]:
# Test 2 : Nombre de voiture sur les routes = embouteillage
# On veut que le phenomène soit penalisé x2 si plus de la moitié de la route occupé par des voitures
# Et x5 les voitures sur les cases départ avec plus de 1 voiture

def nbr_voiture_coef(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        if traffic[i][j].count(1) > len(route[i][j])/2:
                            compt += 5*traffic[i][j].count(1)
                        else:
                            compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] == "Départ":
                        if traffic[i][j][0] > 1:
                            compt += 10*traffic[i][j][0]
                        else:
                            compt += traffic[i][j][0]
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res

def test_2(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture_coef(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res

r = test_2(route_etude, direction_etude, traffic_etude, 30, 60, 10, 1000)
index = sorted(range(len(r)), key=lambda i: r[i])

print(sorted(r))
print([i + 30 for i in index])

100%|██████████| 30/30 [00:21<00:00,  1.42it/s]

[8156268, 8254584, 8273057, 8481426, 8620066, 8668501, 8768660, 8857699, 9037600, 9135052, 9254847, 9355384, 9515353, 9599763, 9766515, 9953581, 10200706, 10237942, 10405894, 10574673, 10726171, 10813658, 10926370, 11088209, 11278565, 11456899, 11608504, 11619753, 11692966, 11876271]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


In [4]:
def r(a):
    if a:
        return 1
    
lis = []

lis.append(r(False))

print(lis)

[None]
